<a href="https://colab.research.google.com/github/jonrtaylor/twitch/blob/master/Production_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade numerox
import numerox as nx
import pandas as pd
import numpy as np
from sklearn import linear_model
data = nx.download('numerai_dataset.zip')
X1_train = data['era1':'era30'].x #training data for partition 1
X2_train = data['era31':'era60'].x #training data for partition 2
X3_train = data['era61':'era90'].x #training data for partition 3
X4_train = data['era91': 'era120'].x #training data for partition 4
X_test = data['validation'].x
y1_train = data['era1':'era30'].df['target'].values
y2_train = data['era31':'era60'].df['target'].values
y3_train = data['era61':'era90'].df['target'].values
y4_train = data['era91':'era120'].df['target'].values
y_test = data['validation'].df['target'].values

Requirement already up-to-date: numerox in /usr/local/lib/python3.6/dist-packages (4.1.7)



numerai_dataset.zip:   0%|          | 0.00/389M [00:00<?, ?B/s]
numerai_dataset.zip:   0%|          | 17.4k/389M [00:00<57:59, 112kB/s]
numerai_dataset.zip:   0%|          | 52.2k/389M [00:00<49:29, 131kB/s]
numerai_dataset.zip:   0%|          | 104k/389M [00:00<40:35, 159kB/s] 
numerai_dataset.zip:   0%|          | 225k/389M [00:00<30:58, 209kB/s]
numerai_dataset.zip:   0%|          | 453k/389M [00:00<23:01, 281kB/s]
numerai_dataset.zip:   0%|          | 923k/389M [00:00<16:45, 385kB/s]
numerai_dataset.zip:   0%|          | 1.88M/389M [00:01<12:01, 536kB/s]
numerai_dataset.zip:   1%|          | 3.58M/389M [00:01<08:33, 749kB/s]
numerai_dataset.zip:   2%|▏         | 5.93M/389M [00:01<06:05, 1.05MB/s]
numerai_dataset.zip:   2%|▏         | 7.97M/389M [00:01<04:23, 1.45MB/s]
numerai_dataset.zip:   3%|▎         | 10.1M/389M [00:01<03:11, 1.97MB/s]
numerai_dataset.zip:   3%|▎         | 12.2M/389M [00:01<02:21, 2.65MB/s]
numerai_dataset.zip:   4%|▎         | 14.3M/389M [00:02<01:47, 3.49MB/

In [ ]:
L1 = linear_model.LinearRegression()
L2 = linear_model.LinearRegression()
L3 = linear_model.LinearRegression()
L4 = linear_model.LinearRegression()

In [ ]:
L1.fit(X1_train, y1_train)
L2.fit(X2_train, y2_train)
L3.fit(X3_train, y3_train)
L4.fit(X4_train, y4_train)
#score function
def correlation_score(y_true, y_pred):
    return np.corrcoef(y_true, y_pred)[0,1]

In [ ]:
L1_preds = L1.predict(X_test) #make predictions on the out-of-sample test set
L2_preds = L2.predict(X_test)
L3_preds = L3.predict(X_test)
L4_preds = L4.predict(X_test)

In [ ]:
L1_score = correlation_score(y_test, L1_preds) #calculate correlation scores on the out-of-sample predictions
L2_score = correlation_score(y_test, L2_preds)
L3_score = correlation_score(y_test, L3_preds)
L4_score = correlation_score(y_test, L4_preds)

In [ ]:
print(L1_score) #each partition's out-of-sample correlation score
print(L2_score)
print(L3_score)
print(L4_score)

0.015131133484415204
0.012834410850005366
0.00992985005947797
0.007843202986115945


In [ ]:
preds = pd.DataFrame(L1_preds)
preds.columns = ['L1_preds']
preds['L2_preds'] = L2_preds
preds['L3_preds'] = L3_preds
preds['L4_preds'] = L4_preds
preds['averaged_preds'] = (preds.L1_preds + preds.L2_preds + preds.L3_preds + preds.L4_preds) / 4 #arithmetic average of each model's out of sample predictions

In [ ]:
averaged_vali_score = correlation_score(y_test, preds.averaged_preds) #averaged out-of-sample correlation score; should be higher than any one model's individual score
print(averaged_vali_score)

0.01615743682941112


In [ ]:
your_name = "UserName"
tournament_name = "nomi" #don't change
prediction_name = "prediction" #don't change
preds1 = pd.DataFrame(data['tournament'].ids)
preds1.columns = ['id']
preds1.set_index("id", inplace=True)
preds1['L1_preds'] = L1.predict(data['tournament'].x)
preds1['L2_preds'] = L2.predict(data['tournament'].x)
preds1['L3_preds'] = L3.predict(data['tournament'].x)
preds1['L4_preds'] = L4.predict(data['tournament'].x)
preds1[prediction_name] = (preds1.L1_preds + preds1.L2_preds + preds1.L3_preds + preds1.L4_preds) / 4
preds1[prediction_name].to_csv(tournament_name + "_" + your_name + "_submission.csv", header=True)

In [ ]:
#public_id = "REPLACEME"
#secret_key = "REPLACEME"
#napi = numerapi.NumerAPI(public_id=public_id, secret_key=secret_key)
#submission_id = napi.upload_predictions(tournament_name + "_" + your_name + "_submission.csv")

In [ ]:
from google.colab import files
files.download(tournament_name + "_" + your_name + "_submission.csv")